In [1]:
import os
from nbed.driver import NbedDriver
import numpy as np

from nbed.ham_builder import HamiltonianBuilder
from openfermion import get_sparse_operator
import scipy as sp
    
from pyscf import gto, scf, fci, cc

/home/lex/anaconda3/envs/embed/lib/python3.9/site-packages/pyscf/lib/misc.py:46: H5pyDeprecationWarning: Using default_file_mode other than 'r' is deprecated. Pass the mode to h5py.File() instead.
  h5py.get_config().default_file_mode = 'a'


In [2]:
# get xyz file for water
notebook_dir = os.getcwd()
projects_dir = os.path.dirname(notebook_dir)
NBed_dir = os.path.dirname(projects_dir)
Test_dir = os.path.join(NBed_dir, 'tests')
mol_dir = os.path.join(Test_dir, 'molecules')

water_xyz_path = os.path.join(mol_dir, 'water.xyz')

In [3]:
geometry_path = os.path.join(mol_dir, 'water.xyz')
# geometry_path = os.path.join(mol_dir, 'H4.xyz')

In [4]:
geometry_path

'/home/lex/Documents/PhD/Nbed/tests/molecules/water.xyz'

In [5]:
with open(geometry_path, 'r') as infile:
    xyz_string = infile.read()
    
print(xyz_string)

3

O   0.00000  0.0000000  0.1653507
H   0.00000  0.7493682 -0.4424329
H   0.00000 -0.7493682 -0.4424329 


In [6]:
# options
geometry = xyz_string
n_active_atoms=1
basis = 'STO-3G'
xc_functional = 'B3LYP'# 'lda, vwn' #'B3LYP'
run_virtual_localization = False

run_fci_emb = False
run_ccsd_emb = False
max_ram_memory = 8_000
_init_huzinaga_rhf_with_mu = False

max_hf_cycles=5000


projector = 'both'
localization = 'ibo' # spade, ibo
occupied_threshold = 0.95
virtual_threshold = 0.95

In [7]:
driver = NbedDriver(geometry = geometry,
                    n_active_atoms=n_active_atoms,
                    basis = basis,
                    xc_functional = xc_functional,
                    run_virtual_localization = run_virtual_localization,
                    run_fci_emb = run_fci_emb,
                    run_ccsd_emb = run_ccsd_emb,
                    max_ram_memory = max_ram_memory,
                    _init_huzinaga_rhf_with_mu = _init_huzinaga_rhf_with_mu,
                    max_hf_cycles=max_hf_cycles,
                    occupied_threshold=occupied_threshold,
                    projector = projector,
                    localization = localization)

 Iterative localization: IB/P4/2x2, 5 iter; Final gradient 6.35e-09


In [8]:
global_DFT_energy = driver._global_rks.e_tot
global_FCI_energy = driver._global_fci.e_tot

print(f'global FCI energy: {global_FCI_energy}')
print(f'global DFT energy: {global_DFT_energy}')

global FCI energy: -75.0153524759469
global DFT energy: -75.27788483404667


In [9]:
qubit_transform = 'jordan_wigner'

    
huz_classical = driver._huzinaga['classical_energy']
rhf_huz = driver._huzinaga["scf"]
qham_huz = HamiltonianBuilder(
                        scf_method=rhf_huz,
                        constant_e_shift=huz_classical,
                        transform=qubit_transform,
                        ).build(n_qubits=None, taper=False)

mu_classical = driver._mu['classical_energy']
rhf_mu = driver._mu["scf"]
qham_mu = HamiltonianBuilder(
                        scf_method=rhf_mu,
                        constant_e_shift=mu_classical,
                        transform=qubit_transform,
                        ).build(n_qubits=None, taper=False)

In [10]:
huz_H_sparse = get_sparse_operator(qham_huz)
huz_e_emb = sp.sparse.linalg.eigsh(huz_H_sparse, k=1, which='SA')[0][0]
huz_e_emb

mu_H_sparse = get_sparse_operator(qham_mu)
mu_e_emb = sp.sparse.linalg.eigsh(mu_H_sparse, k=1, which='SA')[0][0]
mu_e_emb

-75.11213754019393

In [11]:
huz_e_emb

-75.1121355841234

In [12]:
mu_e_emb

-75.11213754019393

In [13]:
print(driver.localized_system.active_MO_inds)
print(driver.localized_system.enviro_MO_inds)

[0 3 4]
[1 2]


In [14]:
print(f'global DFT error: {abs(global_FCI_energy - global_DFT_energy)}')

print(f'mu error: {abs(global_FCI_energy - huz_e_emb)}')
print(f'huz error: {abs(global_FCI_energy - mu_e_emb)}')

global DFT error: 0.2625323580997758
mu error: 0.0967831081764956
huz error: 0.0967850642470296


In [15]:
len(list(qham_huz))

276

In [16]:
len(list(qham_mu))

276